In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats)
import time

In [2]:
pbp2022=pd.read_csv(filepath_or_buffer="PBP Data/2021-22_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [3]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2021-22",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season="2021-22",
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [4]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [5]:
hustle_70percent_gp.nlargest(5,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
37,Blake Griffin,0.989011
344,Kevin Love,0.548879
420,Moritz Wagner,0.518400
192,Garrison Mathews,0.512409
358,Kyle Lowry,0.503790


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [6]:
hustle_70percent_gp.nlargest(5,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
482,Robert Williams III,1.512000
282,Jevon Carter,1.440000
210,Hamidou Diallo,1.369912
92,Cody Martin,1.350000
292,John Konchar,1.335484


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(5,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
195,Gary Payton II,5.465060
401,Matisse Thybulle,5.195455
128,De'Anthony Melton,4.852863
19,Andre Drummond,4.354839
137,Dejounte Murray,4.060465


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(5,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
490,Rudy Gobert,17.865421
518,Steven Adams,16.524138
244,Jakob Poeltl,15.857439
163,Drew Eubanks,15.798347
322,Jusuf Nurkic,15.063830


# The No Fly Zone Award (presented by Dikembe Mutumbo)*

most blocked dunks as the blocking player

In [9]:
misses=pbp2022[pbp2022.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp2022.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
42,Darius Bazley,9
40,Daniel Gafford,7
92,Ivica Zubac,7
128,Karl-Anthony Towns,7
169,OG Anunoby,7
178,Precious Achiuwa,7


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop\_soup for the idea and asw7412 for the sponsor)

In [10]:
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER3_NAME,count
94,Jaren Jackson Jr.,10
9,Andre Drummond,9
39,Daniel Gafford,9
96,Jarrett Allen,9
152,Myles Turner,8


# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears, presented by Gary Payton & Shawn Kemp)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [11]:
#get alley oops
alley_oops=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Alley',na=False)]
#remove missed alley oop attempts
made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
#remove alley oops that have missing 2nd player (self-alley oop or perhaps missing data?)
made_alley_oops_complete=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
made_alley_oops_complete['p1']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
made_alley_oops_complete['p2']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
made_alley_oops_complete.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,p1,p2,count
210,Clint Capela,Trae Young,54
339,Dwight Powell,Luka Doncic,36
248,Darius Garland,Jarrett Allen,35
606,Mike Conley,Rudy Gobert,32
518,John Collins,Trae Young,31
594,Marcus Smart,Robert Williams III,31


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [12]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)
hustle_50percent_gp_ranks.nsmallest(n=5,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
100,D.J. Augustin,0.190000,0.008571,0.025714,0.071429,0.362857,0.658571
527,Terrence Ross,0.190000,0.091429,0.308571,0.111429,0.051429,0.752857
579,Will Barton,0.190000,0.191429,0.314286,0.008571,0.100000,0.804286
176,Eric Gordon,0.437143,0.014286,0.100000,0.257143,0.082857,0.891429
22,Anfernee Simons,0.451429,0.174286,0.042857,0.011429,0.225714,0.905714


In [13]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME']=='Patrick Beverley'].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
457,Patrick Beverley,0.985714,0.1,0.888571,0.014286,0.788571,2.777143


# The "Mr. Fantastic" Award (presented by Shane Battier, the No-Stats All-Star)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [14]:
hustle_50percent_gp_ranks.nlargest(n=5,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
162,Draymond Green,0.925714,0.817143,0.854286,0.911429,0.940000,4.448571
229,Ish Wainright,0.977143,0.671429,0.788571,0.842857,0.925714,4.205714
309,Josh Okogie,0.928571,0.691429,0.928571,0.700000,0.880000,4.128571
378,Luka Garza,0.991429,0.885714,0.705714,0.988571,0.557143,4.128571
202,Goga Bitadze,0.745714,0.942857,0.597143,0.957143,0.882857,4.125714


# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8), presented by Glen "Big Baby" Davis)*

most charges committed (credit to Kdog122025 for the idea)

In [15]:
charges=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
113,Giannis Antetokounmpo,16
171,Jusuf Nurkic,14
167,Julius Randle,12
141,Jaren Jackson Jr.,10
149,Jerami Grant,10
189,Kyle Kuzma,10
196,Luguentz Dort,10
235,Pascal Siakam,10
271,Tobias Harris,10


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [16]:
goaltends=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
34,Clint Capela,13
8,Andre Drummond,11
143,Myles Turner,11
82,JaVale McGee,10
164,Richaun Holmes,10


# The Anti-Verticality Award (presented by Roy Hibbert)*

most 3-point shooting fouls committed (credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea)

In [17]:
third_free_throws=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp2022['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME_x,count
112,Garrett Temple,7
86,Desmond Bane,6
128,Isaac Okoro,6
261,Quentin Grimes,6
314,Tyrese Maxey,6


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

1. Luka Doncic (85.5%)
2. Trae Young (84%)
3. Shai Gilgeous-Alexander (83.8%)
4. Chris Paul (83.4%)
4. James Harden (81.5%)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

1. Wayne Ellington (95.9%)
2. Davis Bertans (95.4%)
3. Garrison Mathews (94.7%)
4. Ben McLemore (94.6%)
5. Nicolas Batum (93%)

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award

Lowest contested rebound percentage, minimum 50% of games played

In [18]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
112,Tyus Jones,0.079
71,Immanuel Quickley,0.089
421,Gary Harris,0.090
26,Trae Young,0.092
168,Aaron Holiday,0.096


In [19]:
reb_w_gp_qualify[reb_w_gp_qualify['PLAYER_NAME']=='Carmelo Anthony'][['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
286,Carmelo Anthony,0.304


alternatively: restricting to players > 6 foot 6 inches in height

In [20]:
player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
158,Ziaire Williams,0.127
47,Furkan Korkmaz,0.131
18,Cedi Osman,0.142
5,Amir Coffey,0.144
41,Duncan Robinson,0.150


# The "Where There's a Will, There's a Way" Award (presented by Dennis Rodman)

Highest contested rebound percentage, minimum 50% of games played

In [21]:
reb_w_gp_qualify.nlargest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
338,Jock Landale,0.608
432,Day'Ron Sharpe,0.603
69,Mitchell Robinson,0.589
138,Jakob Poeltl,0.579
381,Justin Champagnie,0.554
393,Luka Garza,0.554


alternatively: restricting to players < 6 foot 7 inches in height

In [22]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
108,Justin Champagnie,0.554
172,Thanasis Antetokounmpo,0.436
82,Ish Wainright,0.417
114,Kenrich Williams,0.391
117,Kenyon Martin Jr.,0.388


# The Rotation Awards

**value depth over one solitary star. used basketball-reference's play-by-play position percentages as initial baseline for position groupings**

# The Best Guard Rotation Award (sponsored by Buckingham Palace)

# The Best Big Rotation Award (jointly sponsored by Tom Hanks, Cadbury and Sex and the City)

# The Best Wing Rotation Award (co-sponsored by Lou Williams and Magic City)